# Getting Started with Qupy

Welcome to the Qupy backtesting framework! This notebook will walk you through the basics of:
- Loading and preparing data
- Creating a simple strategy
- Running a backtest
- Analyzing results


In [ ]:
# Import necessary libraries
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Qupy imports - updated with interactive plotting
from engine.strategy_base import Strategy, Bar, register_strategy
from engine.context import Context
from engine.data import load_klines_csv
from engine.backtest import run_backtest_json, StrategyOptimizer
from engine.plots import TradingPlots

# Initialize interactive plotting
plotter = TradingPlots(theme="plotly_white", width=900, height=600)

# Set display options
pd.set_option('display.max_rows', 10)

## 1. Loading Data

Let's start by loading some sample data. We'll use the included CAKE/USDT data as an example.

In [ ]:
# Load sample data using proper data loader
df, freq_hint = load_klines_csv('../data/CAKEUSDT.csv')

# Display basic info
print(f"Data shape: {df.shape}")
print(f"Date range: {df['dt_open'].iloc[0]} to {df['dt_open'].iloc[-1]}")
print(f"Frequency hint: {freq_hint}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

## 2. Data Exploration

Before building a strategy, let's explore the data to understand its characteristics.

In [ ]:
# Calculate basic statistics
returns = df['close'].pct_change()

# Use frequency hint to determine periods per day
if freq_hint:
    if 'min' in freq_hint:
        minutes = int(freq_hint.replace('min', ''))
        periods_per_day = 24 * 60 / minutes
    elif 'h' in freq_hint:
        hours = int(freq_hint.replace('h', ''))
        periods_per_day = 24 / hours
    elif 'd' in freq_hint:
        periods_per_day = 1
    else:
        periods_per_day = 96  # Default to 15-min intervals
else:
    periods_per_day = 96  # Default

annualization_factor = np.sqrt(365 * periods_per_day)

stats = {
    'Mean Return': returns.mean(),
    'Std Dev': returns.std(),
    'Sharpe Ratio': returns.mean() / returns.std() * annualization_factor if returns.std() > 0 else 0,
    'Min Return': returns.min(),
    'Max Return': returns.max(),
    'Skewness': returns.skew(),
    'Kurtosis': returns.kurt()
}

print(f"Data frequency: {freq_hint}")
print(f"Periods per day: {periods_per_day}")
print("Return Statistics:")
for key, value in stats.items():
    print(f"{key:15s}: {value:10.6f}")

In [ ]:
# Interactive visualization of price, volume and returns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(
    rows=1, cols=1,
    shared_xaxes=True,
    subplot_titles=('CAKE/USDT Price', 'Volume', 'Returns (%)'),
    vertical_spacing=0.05,
    row_heights=[1]
)

# Price chart
fig.add_trace(
    go.Scatter(
        x=df['dt_close'],
        y=df['close'],
        mode='lines',
        name='Close Price',
        line=dict(color='#1f77b4', width=1.5),
        hovertemplate='<b>CAKE/USDT</b><br>' +
                     'Date: %{x}<br>' +
                     'Price: $%{y:.4f}<br>' +
                     '<extra></extra>'
    ),
    row=1, col=1
)


# Update layout
fig.update_layout(
    title="Interactive CAKE/USDT Data Exploration 📊",
    template="plotly_white",
    width=1000,
    height=700,
    hovermode='x unified'
)

fig.update_yaxes(title_text="Price (USDT)", row=1, col=1)

fig.update_xaxes(title_text="Date", row=3, col=1)

fig.show()


## 3. Building Your First Strategy

Let's create a simple Moving Average Crossover strategy:
- **Buy** when fast MA crosses above slow MA
- **Sell** when fast MA crosses below slow MA

In [ ]:
@register_strategy("simple_ma_crossover")
class SimpleMAStrategy(Strategy):
    """
    Simple Moving Average Crossover Strategy
    
    🔧 HYPERPARAMETERS TO CUSTOMIZE:
    
    • fast_period (5-50): Short-term moving average period
      - Lower values = more sensitive to price changes, more trades
      - Higher values = less sensitive, fewer but potentially stronger signals
      - Try: 5, 10, 15, 20
      
    • slow_period (10-200): Long-term moving average period  
      - Must be > fast_period
      - Lower values = more responsive system
      - Higher values = stronger trend confirmation required
      - Try: 20, 30, 50, 100
      
    • notional (100-50000): Position size in quote currency (USDT)
      - Higher values = larger positions, more risk/reward
      - Lower values = smaller positions, less risk
      - Should be appropriate for your account size
      - Try: 1000, 5000, 10000, 25000
      
    📊 STRATEGY LOGIC:
    - BUY: Fast MA crosses ABOVE slow MA (bullish momentum)
    - SELL: Fast MA crosses BELOW slow MA (bearish momentum)
    - Only trades when no position is open (no averaging down/up)
    """
    
    name = "simple_ma_crossover"
    
    @classmethod
    def param_schema(cls):
        return {
            "fast_period": {"type": "int", "min": 5, "max": 50, "default": 10},
            "slow_period": {"type": "int", "min": 10, "max": 200, "default": 30},
            "notional": {"type": "float", "min": 100, "default": 10_000.0}
        }
    
    def on_init(self, context):
        """Initialize strategy parameters"""
        self.fast_period = int(self.params.get("fast_period", 10))
        self.slow_period = int(self.params.get("slow_period", 30))
        self.notional = float(self.params.get("notional", 10_000.0))
        
        # Validation
        if self.fast_period >= self.slow_period:
            raise ValueError(f"fast_period ({self.fast_period}) must be < slow_period ({self.slow_period})")
        
        context.log("info", f"MA Strategy initialized: Fast={self.fast_period}, Slow={self.slow_period}, Size=${self.notional}")
    
    def on_start(self, context):
        """Called at backtest start"""
        context.log("info", "Strategy started")
    
    def calculate_sma(self, prices: list, period: int) -> float:
        """Calculate Simple Moving Average"""
        if len(prices) < period:
            return 0.0
        return sum(prices[-period:]) / period
        
    def on_bar(self, context, symbol: str, bar: Bar):
        """Main trading logic called each bar"""
        # Need enough data for slow MA
        if context.bar_index < self.slow_period:
            return
            
        # Get historical closes
        closes = context.data.history(symbol, 'close', self.slow_period + 1)
        
        if len(closes) < self.slow_period + 1:
            return
        
        # Calculate moving averages
        fast_ma_current = self.calculate_sma(closes, self.fast_period)
        slow_ma_current = self.calculate_sma(closes, self.slow_period)
        
        # Previous MAs for crossover detection
        fast_ma_prev = self.calculate_sma(closes[:-1], self.fast_period)
        slow_ma_prev = self.calculate_sma(closes[:-1], self.slow_period)
        
        # Detect crossovers
        bullish_cross = (fast_ma_prev <= slow_ma_prev) and (fast_ma_current > slow_ma_current)
        bearish_cross = (fast_ma_prev >= slow_ma_prev) and (fast_ma_current < slow_ma_current)
        
        current_pos = context.position.qty
        
        # Trading logic
        if bullish_cross and current_pos == 0:
            # Buy signal - calculate quantity from notional
            qty = context.size.from_notional(self.notional, bar.close)
            context.buy(qty, reason="MA_Bullish_Crossover", size_mode="qty") # Here is logic to buy
            
            context.log("info", f"🟢 BUY: Fast MA={fast_ma_current:.4f} > Slow MA={slow_ma_current:.4f}, Qty={qty:.4f}")
            
        elif bearish_cross and current_pos > 0:
            # Sell signal
            context.close(reason="MA_Bearish_Crossover")
            
            context.log("info", f"🔴 SELL: Fast MA={fast_ma_current:.4f} < Slow MA={slow_ma_current:.4f}")
        
        # Record indicators for analysis
        context.record("fast_ma", fast_ma_current)
        context.record("slow_ma", slow_ma_current)
        context.record("position", 1 if current_pos > 0 else 0)

    def on_trade_close(self, context, trade):
        """Called when a trade closes - use correct attribute"""
        if hasattr(trade, 'pnl_abs') and trade.pnl_abs is not None:
            context.log("info", f"💰 Trade closed: PnL=${trade.pnl_abs:.2f}")
            context.record("trade_pnl", trade.pnl_abs)



## 4. Running the Backtest with Standardized Engine


In [ ]:
# Import Buy & Hold strategy for benchmarking
from strategies.buy_hold_strategy import BuyAndHoldStrategy
from engine.backtest import run_backtest_json

# Create strategy instance with parameters
strategy_params = {
    "fast_period": 10,
    "slow_period": 30,
    "notional": 10000
}

# Single strategy test with clean JSON output and pretty results
report = run_backtest_json(
    data=df,
    strategy=SimpleMAStrategy(strategy_params),
    symbol="CAKEUSDT",
    initial_cash=100_000,
    fee_bps=10,  # 0.1% fees (10 basis points)
    slippage_bps=1,    # 0.01% slippage (1 basis point)
    pretty_results=1  # Display formatted performance report
)


# Store report for later use
backtest_report = report

In [ ]:
backtest_report['show_all_plots']()

In [ ]:
# 5.1. Flexible Strategy Comparison Demo 🔥


fig1 = report['plot_vs_strategy'](benchmark_strategy='buy_hold', title='MA Strategy vs Buy & Hold')
fig1.show()


In [ ]:
# 6. Interactive Parameter Optimization 🎯

from engine.backtest import StrategyOptimizer

# Create optimizer instance with data and settings
optimizer = StrategyOptimizer(
    data=df,
    symbol="CAKEUSDT", 
    initial_cash=100_000,
    fee_bps=10,  # 0.1% fees
    slippage_bps=1   # 0.01% slippage
)

# Define parameter ranges to test
param_ranges = {
    "fast_period": [5, 10, 15, 20],
    "slow_period": [20, 30, 40, 50], 
    "notional": [10_000]  # Keep position size fixed
}

# Run optimization with clean output
optimization_results = optimizer.optimize_parameters(
    strategy_class=SimpleMAStrategy,
    param_ranges=param_ranges,
    optimization_metric="sharpe_ratio",  # Optimize for risk-adjusted returns
    n_best=5,  # Show top 5 results
    verbose=False  # Hide verbose output
)


optimizer.show_optimization_results(optimization_results, pretty_results=1)


In [ ]:

# 🎯 HYPERPARAMETER: Control visualization output
plot_all_equities = True  # Set to False to show only heatmap


heatmap_fig = optimizer.plot_optimization_heatmap(
    optimization_results,
    param1="fast_period",
    param2="slow_period", 
    metric="sharpe_ratio"
)
if heatmap_fig:
    heatmap_fig.show()


In [ ]:
strategies_fig = optimizer.plot_optimization_strategies(
    optimization_results,
    SimpleMAStrategy,  # Fixed: positional argument instead of keyword
    n_strategies=5,
    title="Top 5 MA Strategy Parameter Combinations"
)
if strategies_fig:
    strategies_fig.show()
